<h1 align="center">Youtube data construction</h1>
<h2 align=center> What did I watch on youtube ? </h2>
<br>
<center>The purpose of this script is to create differents dataframe containing informations about my youtube watch history.</center>
<center>A second notebook will be used to analyse this data.</center>

The purpose here is to : 
* fetch and clean the download my-youtube-watch-history received from google service, 
* web scraping of video details and channel infos from the Youtube API. 

In [17]:
import pandas as pd
from googleapiclient.discovery import build
from datetime import time

<h2 align="center">Raw Data</h2>

In [44]:
#file path
raw_data = pd.read_json('***/watch-history.json')

In [3]:
raw_data.to_excel('raw_data.xlsx')

<h2 align="center">Cleaning of data</h2>

<h3> columns cleaning </h3>

In [52]:
wt = raw_data.copy()
wt = wt.drop(['header','products','activityControls','description'],axis=1)

In [20]:
wt.head(3)

,title,titleUrl,subtitles,time,details
0,Vous avez regardé Récupérer ses Données You...,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",2024-03-31T14:22:02.374Z,NaN
1,Vous avez regardé FR_fr_1019_Spring_Instream_Y...,https://www.youtube.com/watch?v=JR9tdqcj0-4,NaN,2024-03-31T14:21:52.118Z,[{'name': 'Des annonces Google'}]
2,Vous avez regardé BeautifulSoup + Requests | W...,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31T13:52:12.369Z,NaN


<h3> columns organization </h3>

In [53]:
#Cleaning Title
wt['title'] = wt['title'].str.replace('Vous avez regardé ','')

#Fetch video-ID from the url and place it on a different columns
wt['video_ID'] = wt['titleUrl'].str.split('=').str[1]

#Fetch the date and time in different columns and set them to datetime type
wt[['date','time']] = wt['time'].str.split('T',expand=True)
wt['date'] = pd.to_datetime(wt['date'])
wt['time'] = wt['time'].str.split('.').str[0]
wt['time'] = wt['time'].str.replace('Z','')
wt['time'] = pd.to_datetime(wt['time'],format='%H:%M:%S')
wt.head(3)

,title,titleUrl,subtitles,time,details,video_ID,date
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",1900-01-01 14:22:02,NaN,4Vo6kF4jWn8,2024-03-31
1,FR_fr_1019_Spring_Instream_YT_16x9_10_C_414081...,https://www.youtube.com/watch?v=JR9tdqcj0-4,NaN,1900-01-01 14:21:52,[{'name': 'Des annonces Google'}],JR9tdqcj0-4,2024-03-31
2,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",1900-01-01 13:52:12,NaN,bargNl2WeN4,2024-03-31


In [57]:
wt['hour'] = wt["time"].dt.hour
wt['minute'] = wt["time"].dt.minute
wt = wt.drop(['time'],axis=1)

<h3>Filter the advertisement in a different dataframe</h3>

* Dataframe tableau_pub : only Google advertisement
* Dataframe working_table (wt) : extract the advertisement

In [58]:
tableau_pub = wt.copy()
tableau_pub = wt[wt['subtitles'].isnull()&wt['details'].notnull()]
tableau_pub = tableau_pub.drop(['video_ID'],axis=1)
tableau_pub.head(3)

,title,titleUrl,subtitles,details,date,hour,minute
1,FR_fr_1019_Spring_Instream_YT_16x9_10_C_414081...,https://www.youtube.com/watch?v=JR9tdqcj0-4,NaN,[{'name': 'Des annonces Google'}],2024-03-31,14,21
3,02 ResponsableProjetCybersecuriteSI 16 9,https://www.youtube.com/watch?v=wKLFC8Uxvtc,NaN,[{'name': 'Des annonces Google'}],2024-03-31,13,52
4,20221219 HELLOFRESH PEACE OF MIND V5 ssST,https://www.youtube.com/watch?v=xDlM2D1LGDQ,NaN,[{'name': 'Des annonces Google'}],2024-03-31,13,51


In [59]:
wt = wt[wt['subtitles'].notnull()]
wt = wt.drop(['details'],axis=1)
wt.head()

,title,titleUrl,subtitles,video_ID,date,hour,minute
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",4Vo6kF4jWn8,2024-03-31,14,22
2,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",bargNl2WeN4,2024-03-31,13,52
5,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,"[{'name': 'Alex The Analyst', 'url': 'https://...",q-kbzWjyPak,2024-03-31,13,43
7,Building an Automated File Sorter in File Expl...,https://www.youtube.com/watch?v=gs0FNQR0njI,"[{'name': 'Alex The Analyst', 'url': 'https://...",gs0FNQR0njI,2024-03-31,13,39
8,FREE Data Analyst Bootcamp!!,https://www.youtube.com/watch?v=rGx1QNdYzvs,"[{'name': 'Alex The Analyst', 'url': 'https://...",rGx1QNdYzvs,2024-03-31,13,39


<h3> Fetch data from the subtitles columns </h3>

In [61]:
author = []
channel_url = []
for element in wt['subtitles']:
    author.append(element[0]["name"])
    channel_url.append(element[0]["url"])
wt['creator']=author
wt['channel_url']=channel_url

In [62]:
wt.head(3)

,title,titleUrl,subtitles,video_ID,date,hour,minute,creator,channel_url
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",4Vo6kF4jWn8,2024-03-31,14,22,Labo Des Réseaux,https://www.youtube.com/channel/UCMJEpQVv0p3qz...
2,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",bargNl2WeN4,2024-03-31,13,52,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...
5,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,"[{'name': 'Alex The Analyst', 'url': 'https://...",q-kbzWjyPak,2024-03-31,13,43,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...


<h3> Index reset </h3>

In [63]:
wt.index=range(len(wt))
wt = wt.drop(['subtitles'],axis=1)
wt.head(3)

,title,titleUrl,video_ID,date,hour,minute,creator,channel_url
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,4Vo6kF4jWn8,2024-03-31,14,22,Labo Des Réseaux,https://www.youtube.com/channel/UCMJEpQVv0p3qz...
1,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,bargNl2WeN4,2024-03-31,13,52,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...
2,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,q-kbzWjyPak,2024-03-31,13,43,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...


<h3>Fetch channel_id from the channel_url </h3>

In [64]:
wt['channel_id']= wt['channel_url'].str.replace('https://www.youtube.com/channel/','')
wt.head(3)

,title,titleUrl,video_ID,date,hour,minute,creator,channel_url,channel_id
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,4Vo6kF4jWn8,2024-03-31,14,22,Labo Des Réseaux,https://www.youtube.com/channel/UCMJEpQVv0p3qz...,UCMJEpQVv0p3qzJnQ9IuiiMA
1,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,bargNl2WeN4,2024-03-31,13,52,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...,UC7cs8q-gJRlGwj4A8OmCmXg
2,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,q-kbzWjyPak,2024-03-31,13,43,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...,UC7cs8q-gJRlGwj4A8OmCmXg


<h2 align=center> Recuperation channels data from Youtube API </h2>

* identification channels where I watched more than 10 videos
* convert there channel_id to a list
* pass the list to the youtube API and scrap their data

In [65]:
count = wt.channel_id.value_counts()
count = count[count > 10]
len(count)

312

In [66]:
channel_scraping = pd.DataFrame(data=count)
channel_scraping = channel_scraping.reset_index()

In [67]:
#put your api key '****'
api_key ='****'
youtube= build('youtube', 'v3', developerKey=api_key)

In [68]:
channel_ids = channel_scraping['channel_id'].tolist()

In [69]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    
    for i in range(0,len(channel_ids),50):                  
        request = youtube.channels().list(
            part="snippet,contentDetails,statistics,id",
            id = ','.join(channel_ids[i:i+50]))
        response = request.execute()

        for i in range(len(response['items'])):
            data =dict(Channel_name = response['items'][i]['snippet']['title'],
                       channel_id = response['items'][i]['id'], 
                       Channel_date = response['items'][i]['snippet']['publishedAt'],
                       Subscribers = response['items'][i]['statistics']['subscriberCount'],
                       Views = response['items'][i]['statistics']['viewCount'],
                       country = response['items'][i]['snippet'].get('country'),
                       Total_videos = response['items'][i]['statistics']['videoCount'])        
            all_data.append(data)
    
    return all_data

In [70]:
channel_statitics = get_channel_stats(youtube, channel_ids)
channel_data = pd.DataFrame(channel_statitics)

In [71]:
data_channel = pd.merge(channel_data, channel_scraping, how='inner',on='channel_id')
data_channel = data_channel.sort_values('count',ascending=False)
data_channel

,Channel_name,channel_id,Channel_date,Subscribers,Views,country,Total_videos,count
31,CloudKid,UCSa8IUd1uEjlREMa21I3ZPQ,2013-07-30T20:10:22Z,5330000,2967421662,US,2752,1432
0,ImagineDragonsVEVO,UCpx_k19S2vUutWUUM9qmXEg,2012-01-23T18:04:00Z,9530000,14866408443,None,226,401
5,Adam Jensen,UChRJQ2QyKsY1EbBqM8GzzRg,2011-10-07T00:27:55Z,117000,89401192,US,71,330
43,McSkyz,UC4OjGShlHwNNd6jQ23GVJ5A,2011-07-18T09:49:59Z,1740000,280463607,FR,286,234
12,twenty one pilots,UCBQZwaNPFfJ1gZ1fLZpAEGw,2009-09-05T05:57:18Z,13400000,5778309370,US,204,193
...,...,...,...,...,...,...,...,...
256,SrJinxed,UCjKOPG-HhqMjdqp5pJufoRQ,2019-12-03T21:45:24.487117Z,6270000,5579894069,PA,125,11
253,Anthony Crazy,UCwlLTQ2yiKu0GCwNK70-pww,2018-04-02T20:50:23Z,373000,173596730,FR,69,11
251,POPSLAY,UCZNu2DGg9phKQUzUywxawAA,2021-04-18T17:00:25.318517Z,128000,12211123,FR,82,11
250,Laura Laune,UCb32WqbvB979c0rDifGZVvA,2013-08-24T13:27:03Z,291000,79528099,FR,88,11


In [72]:
# cleanning datatype : numeric
numeric_cols = ['Subscribers','Views','Total_videos']
data_channel[numeric_cols] = data_channel[numeric_cols].apply(pd.to_numeric, errors='coerce')

#cleaning datatype: datetime
data_channel['date'] = data_channel['Channel_date'].str.split('T').str[0]
data_channel['date'] = pd.to_datetime(data_channel['date'])
data_channel = data_channel.drop(['Channel_date'],axis=1)

#result
data_channel.dtypes

Channel_name            object
channel_id              object
Subscribers              int64
Views                    int64
country                 object
Total_videos             int64
count                    int64
date            datetime64[ns]
dtype: object

In [73]:
data_channel.head(3)

,Channel_name,channel_id,Subscribers,Views,country,Total_videos,count,date
31,CloudKid,UCSa8IUd1uEjlREMa21I3ZPQ,5330000,2967421662,US,2752,1432,2013-07-30
0,ImagineDragonsVEVO,UCpx_k19S2vUutWUUM9qmXEg,9530000,14866408443,None,226,401,2012-01-23
5,Adam Jensen,UChRJQ2QyKsY1EbBqM8GzzRg,117000,89401192,US,71,330,2011-10-07


<h2 align = center> Recuperation datas from all my videos history </h2>

In [74]:
count_video = wt.video_ID.value_counts()
len(count_video)

16648

In [76]:
video_scraping = pd.DataFrame(data=count_video)
video_scraping = video_scraping.reset_index()
video_scraping.head(3)

,video_ID,count
0,EgwQG3MYp3o,67
1,i4AXVFggAnk,64
2,L3wKzyIN1yk,63


In [77]:
video_ids = video_scraping['video_ID'].tolist()

In [78]:
def get_video_details(youtube, video_ids):
    all_videos_stats = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part = 'snippet, statistics',
            id=','.join(video_ids[i:i+50]))
        response = request.execute()

        for video in response['items']:
            video_stats = dict(title = video['snippet']['title'],
                                Published_date = video['snippet']['publishedAt'],
                                Views = video['statistics'].get('viewCount'),
                                Like = video['statistics'].get('likeCount'),
                                Comments = video['statistics'].get('commentCount'),
                                Videocategory = video['snippet']['categoryId']
                              )
            all_videos_stats.append(video_stats)

    return all_videos_stats

In [79]:
video_statitics = get_video_details(youtube, video_ids)
video_data = pd.DataFrame(video_statitics)
video_data.head(3)

,title,Published_date,Views,Like,Comments,Videocategory
0,Silent Child - F**k You (Lyric Video),2020-08-26T15:27:16Z,42252535,597052,3340,10
1,Stela Cole - Love Like Mine,2020-09-25T15:27:08Z,17830466,139994,933,10
2,Rag'n'Bone Man - Human (Official Video),2016-07-21T11:00:00Z,1880302234,12194902,247774,10


In [80]:
data_video = pd.merge(video_data, wt, how='inner',on='title')
data_video.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22317 entries, 0 to 22316
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           22317 non-null  object        
 1   Published_date  22317 non-null  object        
 2   Views           22316 non-null  object        
 3   Like            22012 non-null  object        
 4   Comments        22154 non-null  object        
 5   Videocategory   22317 non-null  object        
 6   titleUrl        22317 non-null  object        
 7   video_ID        22317 non-null  object        
 8   date            22317 non-null  datetime64[ns]
 9   hour            22317 non-null  int32         
 10  minute          22317 non-null  int32         
 11  creator         22317 non-null  object        
 12  channel_url     22317 non-null  object        
 13  channel_id      22317 non-null  object        
dtypes: datetime64[ns](1), int32(2), object(11)
memory usag

In [81]:
# cleanning datatype : numeric
numeric_cols = ['Views','Like','Comments']
data_video[numeric_cols] = data_video[numeric_cols].apply(pd.to_numeric, errors='coerce')

#cleaning datatype: datetime
data_video['Published_date'] = data_video['Published_date'].str.split('T').str[0]
data_video['Published_date'] = pd.to_datetime(data_video['Published_date'])

#result
data_video.dtypes

title                     object
Published_date    datetime64[ns]
Views                    float64
Like                     float64
Comments                 float64
Videocategory             object
titleUrl                  object
video_ID                  object
date              datetime64[ns]
hour                       int32
minute                     int32
creator                   object
channel_url               object
channel_id                object
dtype: object

In [85]:
data_video.Videocategory.value_counts()

Videocategory
10    6152
24    5255
22    4211
23    1914
20    1201
27    1000
1      869
26     558
28     356
25     345
17     230
19     127
15      53
2       26
29      13
44       7
Name: count, dtype: int64

In [91]:
my_categories = {'10':'Music','24':'Entertainment','22':'People&Blog','23':'Comedy','20':'Gaming','27':'Education',
                 '1':'Film&Animation','26':'HowTo&Style','28':'Science&Technology','25':'News&Politics','17':'Sports',
                '19':'Travel&Events','15':'Pets&Animal','2':'Autos','29':'NonProfits&Activism','44':'Trailers'}

In [93]:
data_video['Videocategory'] = data_video['Videocategory'].map(my_categories)

<h2 align='center'> Save Data to Excel </h2>

In [94]:
with pd.ExcelWriter ("youtube_data.xlsx") as writer :
    wt.to_excel(writer, sheet_name='working_data',index=False)
    tableau_pub.to_excel(writer, sheet_name='pub_data', index=False)
    data_channel.to_excel(writer, sheet_name='channel_data',index=False)
    data_video.to_excel(writer,sheet_name='video_data',index=False)